In [41]:
# 데이터 로드
import pandas as pd

df = pd.read_csv("daum_movie_review.csv")

df.head()

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [42]:
df.title.value_counts()

title
신과함께      4947
택시운전사     2322
인피니티 워    2042
범죄도시      1939
곤지암       1547
라라랜드      1150
코코         778
Name: count, dtype: int64

In [43]:
# 훈련 데이터, 테스트 데이터 분리 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.review, df.title, random_state=42)

print('train size:' , len(X_train))
print('train size:' , len(X_test))

train size: 11043
train size: 3682


In [45]:
from konlpy.tag import Okt
okt = Okt()

print(okt.morphs(X_train[1]))
print(okt.nouns(X_train[1]))

['몰입', '할수밖에', '없다', '.', '어렵게', '생각', '할', '필요없다', '.', '내', '가', '전투', '에', '참여', '한', '듯', '손', '에', '땀', '이남', '.']
['몰입', '생각', '내', '전투', '참여', '듯', '손', '땀', '이남']


In [46]:
# 명사를 tfidf 토크나이저로 지정해 로지스틱 회귀 분석 실행 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000) # 반복율 1000, 디폴트 100
clf.fit(X_train_tfidf, y_train)

print('훈련 정확도: {:3}'.format(clf.score(X_train_tfidf,y_train)))

print('테스트 정확도: {:3}'.format(clf.score(X_test_tfidf,y_test)))


C:\Anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


훈련 정확도: 0.7584895408856289
테스트 정확도: 0.6852254209668658


In [48]:
# 테스트 데이터 예측 

print('실제 영화 제목, 예측한 제목, 리뷰내용')

for content in zip(y_test[:10], clf.predict(X_test_tfidf[:10]), X_test[:10]):
    print(content)


실제 영화 제목, 예측한 제목, 리뷰내용
('인피니티 워', '인피니티 워', '누가 죽는다는겁니까? 마블 영화는  한번도 본적이 없어서  스파이더맨3가 마지막')
('신과함께', '신과함께', '재밋습니다')
('범죄도시', '택시운전사', '윤ㄱㅖ상 양옆의 똘마니들은 누군가요 진짜 미친놈 같음ㅎㄷㄷ')
('인피니티 워', '인피니티 워', '완전 허무 마지막 컷에서 타노스는 돌아온다 ㅋㅋㅋㅋ 개봉전 떠돌던 아이언맨과 캡틴은 살았슴.. 다른 배우들이 문제지.... 쩝')
('범죄도시', '신과함께', '스토리는 없음. 지루하지는 않았으나 7점 정도가 적당한거 같은데....타짜하고 추격자 이상인지는 모르겠음')
('코코', '코코', "이제 막 6살된 딸이랑 보러갔습니다. 딸에게는 다소 어려운 세계관이었는지, 보는 내내 여러 질문들을 하기는 했습니다. 딸도 잘 이해는 못하지만, 재미있게 봤다고 하고, 저 역시 약간의 감동과 재미가 남는 애니메이션이었습니다. 딸아이가 영화보고 나오자 마자 '기억해줘'를 틀어달라고 하네요 ㅎㅎ")
('범죄도시', '신과함께', '재밌어요')
('인피니티 워', '신과함께', '잼있어 빵점 준다')
('라라랜드', '라라랜드', '작년에 본 영화중에 가장 행복했던 영화.. 지인이 라라랜드 보러가자고 해서 무슨 영화 제목이 그러냐고 했다가 지금까지 OST 듣고 다닌다지.. 개인적으로 엔딩까지 훌륭했던 영화')
('신과함께', '신과함께', '내용은 생각보다 넘 재미없내요 노잼 노잼 영상은 굿-')


In [49]:
#품사를 사용하여 예측 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

tfidf = TfidfVectorizer(tokenizer=okt.morphs, max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)  # 반복율 1000, 디폴트 100
clf.fit(X_train_tfidf, y_train)

print("훈련 정확도: {:3}".format(clf.score(X_train_tfidf, y_train)))

print("테스트 정확도: {:3}".format(clf.score(X_test_tfidf, y_test)))

C:\Anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


훈련 정확도: 0.7790455492166983
테스트 정확도: 0.6936447582835416


In [50]:
# 품사 표시 
print(okt.pos(X_train[1]))

[('몰입', 'Noun'), ('할수밖에', 'Verb'), ('없다', 'Adjective'), ('.', 'Punctuation'), ('어렵게', 'Adjective'), ('생각', 'Noun'), ('할', 'Verb'), ('필요없다', 'Adjective'), ('.', 'Punctuation'), ('내', 'Noun'), ('가', 'Josa'), ('전투', 'Noun'), ('에', 'Josa'), ('참여', 'Noun'), ('한', 'Determiner'), ('듯', 'Noun'), ('손', 'Noun'), ('에', 'Josa'), ('땀', 'Noun'), ('이남', 'Noun'), ('.', 'Punctuation')]


In [51]:
# 명사, 동사, 형용사만 선별 
def twit_tokenizer(text):
    target_tags = ['Nouns', 'Verb', 'Adjective']
    result = []
    for word, tag in okt.pos(text, norm=True, stem=True):
        if tag in target_tags:
            result.append(word)
        
    return result

In [52]:
tfidf = TfidfVectorizer(tokenizer=okt.morphs, max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

clf = LogisticRegression(max_iter=1000)  # 반복율 1000, 디폴트 100
clf.fit(X_train_tfidf, y_train)

print("훈련 정확도: {:3}".format(clf.score(X_train_tfidf, y_train)))

print("테스트 정확도: {:3}".format(clf.score(X_test_tfidf, y_test)))


C:\Anaconda\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


훈련 정확도: 0.7790455492166983
테스트 정확도: 0.6936447582835416


In [53]:
# 테스트 데이터 예측

print("실제 영화 제목, 예측한 제목, 리뷰내용")

for content in zip(y_test[:10], clf.predict(X_test_tfidf[:10]), X_test[:10]):
    print(content)


실제 영화 제목, 예측한 제목, 리뷰내용
('인피니티 워', '인피니티 워', '누가 죽는다는겁니까? 마블 영화는  한번도 본적이 없어서  스파이더맨3가 마지막')
('신과함께', '인피니티 워', '재밋습니다')
('범죄도시', '인피니티 워', '윤ㄱㅖ상 양옆의 똘마니들은 누군가요 진짜 미친놈 같음ㅎㄷㄷ')
('인피니티 워', '인피니티 워', '완전 허무 마지막 컷에서 타노스는 돌아온다 ㅋㅋㅋㅋ 개봉전 떠돌던 아이언맨과 캡틴은 살았슴.. 다른 배우들이 문제지.... 쩝')
('범죄도시', '신과함께', '스토리는 없음. 지루하지는 않았으나 7점 정도가 적당한거 같은데....타짜하고 추격자 이상인지는 모르겠음')
('코코', '코코', "이제 막 6살된 딸이랑 보러갔습니다. 딸에게는 다소 어려운 세계관이었는지, 보는 내내 여러 질문들을 하기는 했습니다. 딸도 잘 이해는 못하지만, 재미있게 봤다고 하고, 저 역시 약간의 감동과 재미가 남는 애니메이션이었습니다. 딸아이가 영화보고 나오자 마자 '기억해줘'를 틀어달라고 하네요 ㅎㅎ")
('범죄도시', '신과함께', '재밌어요')
('인피니티 워', '인피니티 워', '잼있어 빵점 준다')
('라라랜드', '라라랜드', '작년에 본 영화중에 가장 행복했던 영화.. 지인이 라라랜드 보러가자고 해서 무슨 영화 제목이 그러냐고 했다가 지금까지 OST 듣고 다닌다지.. 개인적으로 엔딩까지 훌륭했던 영화')
('신과함께', '신과함께', '내용은 생각보다 넘 재미없내요 노잼 노잼 영상은 굿-')
